# Naive Bayes による未知の文書の判定
可視化を行います。


In [ ]:
## 必要なライブラリの読み込み
import sys
import os
import re
import math
import glob
import pandas as pd

テキストの前処理で作成した処理後（単語分割後）のファイルを使い、どちらのカテゴリ(H1,H2)に、どの単語が、何回出現したかをカウントします。Pandasを利用することで、簡単に集計できます。

In [ ]:
### 教師データ：カテゴリ H1 H2 ごとの重複ありでの全部の単語が入る
### 列名はデータにはなく（header=None）、"W"という名前の列名を付ける。
raw_words = {}
raw_words['H1'] = pd.read_table('mlit.txt',encoding='utf-8',header=None,names=('W'))
raw_words['H2'] = pd.read_table('env.txt',encoding='utf-8',header=None,names=('W'))

### カテゴリ H1 H2 ごと、各単語の出現回数
word_count = {}
word_count['H1'] = raw_words['H1']['W'].value_counts()
word_count['H2'] = raw_words['H2']['W'].value_counts()   ### word_count['H1'].get('事業',0) 等で回数を使える

### カテゴリ H1 H2 両方を通しての全部の出現単語（重複無し）
all_words = pd.concat([raw_words['H1'],raw_words['H2']])['W'].unique()


前処理で作ったチェック用データに対して、ナイーブベイズフィルタによるカテゴリ判定を行います。<br>
どちらになるかは五分五分（0.5）の事前確率からスタートし、

In [ ]:
check_words = {}
check_words['Test1'] = pd.read_table('mlit_check.txt',encoding='utf-8',header=None,names=('W'))
check_words['Test2'] = pd.read_table('env_check.txt',encoding='utf-8',header=None,names=('W'))

posterior = { 'H1': math.log(0.5), 'H2': math.log(0.5) }
for w in check_words['Test1']['W']:
    for c in ['H1', 'H2']:
        posterior[c] += math.log( float(word_count[c].get(w,0) + 1) / ( len(raw_words[c]) + 2 ) )

print("H1 = " + str(posterior['H1']) + " ,H2 = " + str(posterior['H2']) + "\n")
if posterior['H1'] > posterior['H2']:
    print("Test1 is classified to H1\n" )
else:
    print("Test1 is classified to H2\n" )

posterior = { 'H1': math.log(0.5), 'H2': math.log(0.5) }
for w in check_words['Test2']['W']:
    for c in ['H1', 'H2']:
        posterior[c] += math.log( float(word_count[c].get(w,0) + 1) / ( len(raw_words[c]) + 2 ) )
    
print("H1 = " + str(posterior['H1']) + " ,H2 = " + str(posterior['H2']) + "\n")
if posterior['H1'] > posterior['H2']:
    print("Test2 is classified to H1\n" )
else:
    print("Test2 is classified to H2\n" )